### **Data Mining Using SQL**

<font color="red">File access required:</font> In Colab this notebook requires first uploading files **Shop.csv** and **Movies.csv** using the *Files* feature in the left toolbar. If running the notebook on a local computer, simply ensure these files are in the same workspace as the notebook.

In [2]:
!pip install prettytable==0.7.2
!pip install ipython-sql

In [3]:
# Set-up
%load_ext sql
%sql sqlite://
import pandas as pd

In [4]:
# Load tables from CSV files
# Shop
with open('Shop.csv') as f: Shop = pd.read_csv(f, index_col=0)
%sql drop table if exists Shop;
%sql --persist Shop
# Movies
with open('Movies.csv') as f: Movies = pd.read_csv(f, index_col=0)
%sql drop table if exists Movies;
%sql --persist Movies

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted movies'

## Shopping dataset - frequent item-sets

In [5]:
%%sql
select * from Shop

 * sqlite://
Done.


TID,item
1,milk
1,eggs
1,juice
2,milk
2,juice
2,cookies
3,eggs
3,chips
4,milk
4,eggs


### Frequent item-sets of two

#### All pairs of items occurring together

In [6]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item

 * sqlite://
Done.


item1,item2
eggs,milk
eggs,juice
juice,milk
juice,milk
cookies,milk
cookies,juice
chips,eggs
eggs,milk
juice,milk
cookies,milk


#### Now with number of transactions they occur together in

In [7]:
%%sql
select T1.item as item1, T2.item as item2, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item

 * sqlite://
Done.


item1,item2,count()
chips,cookies,1
chips,eggs,1
chips,juice,1
chips,milk,1
cookies,juice,2
cookies,milk,2
eggs,juice,1
eggs,milk,2
juice,milk,3


#### Total number of transactions

In [8]:
%%sql
select count(distinct TID) from Shop

 * sqlite://
Done.


count(distinct TID)
5


#### Put it together for frequent item-sets

In [9]:
support = .3
print(support)

0.3


In [10]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item1,item2
cookies,juice
cookies,milk
eggs,milk
juice,milk


### Frequent item-sets of three

#### All triples of items occurring together

In [11]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item

 * sqlite://
Done.


item1,item2,item3
eggs,juice,milk
cookies,juice,milk
cookies,juice,milk
chips,juice,milk
chips,cookies,milk
chips,cookies,juice


#### Now with number of transactions they occur together in

In [12]:
%%sql
select distinct T1.item as item1, T2.item as item2, T3.item as item3, count()
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


item1,item2,item3,count()
chips,cookies,juice,1
chips,cookies,milk,1
chips,juice,milk,1
cookies,juice,milk,2
eggs,juice,milk,1


#### Put it together for frequent item-sets

In [13]:
support = .3
print(support)

0.3


In [14]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item1,item2,item3
cookies,juice,milk


### <font color = 'green'>**Your Turn - Movies dataset frequent item-sets**</font>

In [15]:
%%sql
select * from Movies limit 15

 * sqlite://
Done.


TID,item
145755,The Fault in Our Stars
145755,Boyhood
145755,Big Hero 6
145755,The Imitation Game
145755,Inside Out
117070,Inside Out
239040,Inside Out
177796,Gone Girl
177796,Magic Mike XXL
126875,The Imitation Game


In [16]:
%%sql
select count(distinct TID) as numUsers,
       count(distinct item) as numMovies
from Movies

 * sqlite://
Done.


numUsers,numMovies
1382,123


#### Mine for frequent item-sets of three and four items in the Movies dataset. Find a single support threshold where the number of frequent item-sets of three items is more than 10 but less than 20, and the number of frequent item-sets of four items is more than 0.

In [58]:
support = 0.03
print(support)

0.03


In [59]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, item3, numtrans);
insert into Frequent
select T1.item, T2.item, T3.item, count()
from Movies T1, Movies T2, Movies T3
where T1.TID = T2.TID
and T2.TID = T3.TID
and T1.item < T2.item
and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Movies) > :support;

select count(*) from Frequent;
select * from Frequent

 * sqlite://
Done.
Done.
14 rows affected.
Done.
Done.


item1,item2,item3,numtrans
Big Hero 6,Boyhood,Gone Girl,56
Big Hero 6,Boyhood,The Imitation Game,57
Big Hero 6,Fury,Gone Girl,43
Big Hero 6,Fury,The Imitation Game,44
Big Hero 6,Gone Girl,Inside Out,85
Big Hero 6,Gone Girl,The Imitation Game,119
Big Hero 6,Inside Out,The Imitation Game,102
Boyhood,Fury,Gone Girl,43
Boyhood,Fury,The Imitation Game,43
Boyhood,Gone Girl,Inside Out,54


In [60]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, item3, item4, numtrans);
insert into Frequent
select T1.item, T2.item, T3.item, T4.item, count()
from Movies T1, Movies T2, Movies T3, Movies T4
where T1.TID = T2.TID
and T2.TID = T3.TID
and T3.TID = T4.TID
and T1.item < T2.item
and T2.item < T3.item
and T3.item < T4.item
group by T1.item, T2.item, T3.item, T4.item
having 1.0*count() / (select count(distinct TID) from Movies) > :support;

select count(*) from Frequent;
select * from Frequent

 * sqlite://
Done.
Done.
3 rows affected.
Done.
Done.


item1,item2,item3,item4,numtrans
Big Hero 6,Boyhood,Gone Girl,The Imitation Game,46
Big Hero 6,Gone Girl,Inside Out,The Imitation Game,59
Boyhood,Gone Girl,Inside Out,The Imitation Game,42


## Shopping dataset - association rules

### Association rules with one item on the left-hand side

In [18]:
support = .5
confidence = .5
print(support, confidence)

0.5 0.5


#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules; include number of transactions

In [19]:
%%sql
select item, count() as numtrans
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### Do the same query but save the result in a table called "Frequents"

In [20]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
3 rows affected.
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [21]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans as LHS_count, count() as both
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

 * sqlite://
Done.


LHS,RHS,LHS_count,both
eggs,chips,3,1
eggs,juice,3,1
eggs,milk,3,2
juice,chips,3,1
juice,cookies,3,2
juice,eggs,3,1
juice,milk,3,3
milk,chips,4,1
milk,cookies,4,2
milk,eggs,4,2


#### Add check for specified confidence to get final result

In [22]:
%%sql
select T1.item as LHS, T2.item as RHS
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item
having 1.0*count() / F.numtrans > :confidence

 * sqlite://
Done.


LHS,RHS
eggs,milk
juice,cookies
juice,milk
milk,juice


### Association rules with two items on the left-hand side

In [23]:
support = .5
confidence = .5
print(support, confidence)

0.5 0.5


#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [24]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
1 rows affected.
Done.


item1,item2,numtrans
juice,milk,3


#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [25]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


LHS1,LHS2,RHS,numtrans,count()
juice,milk,chips,3,1
juice,milk,cookies,3,2
juice,milk,eggs,3,1


#### Add check for specified confidence to get final result

In [26]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / F.numtrans > :confidence

 * sqlite://
Done.


LHS1,LHS2,RHS
juice,milk,cookies


## Shopping dataset - association rules with lift instead of confidence

#### Compute overall frequency of each item -- needed for lift calculation

In [27]:
%%sql
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item

 * sqlite://
Done.


item,1.0*count() / (select count(distinct TID) from Shop)
chips,0.4
cookies,0.4
eggs,0.6
juice,0.6
milk,0.8


#### Do the same query but save the result in a table called "ItemFreq"

In [28]:
%%sql
drop table if exists ItemFreq;
create table ItemFreq(item, freq);
insert into ItemFreq
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item;
select * from ItemFreq

 * sqlite://
Done.
Done.
5 rows affected.
Done.


item,freq
chips,0.4
cookies,0.4
eggs,0.6
juice,0.6
milk,0.8


### Association rules with one item on the left-hand side

In [29]:
support = .5
print(support)

0.5


#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions.

In [30]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
3 rows affected.
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [31]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

 * sqlite://
Done.


LHS,RHS,numtrans,count()
eggs,chips,3,1
eggs,juice,3,1
eggs,milk,3,2
juice,chips,3,1
juice,cookies,3,2
juice,eggs,3,1
juice,milk,3,3
milk,chips,4,1
milk,cookies,4,2
milk,eggs,4,2


#### Add check for lift > 1 to get final result

In [32]:
%%sql
select T1.item as LHS, T2.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Frequent F, ItemFreq I
where T1.item = F.item
and T1.TID = T2.TID
and T2.item = I.item
and T1.item != T2.item
group by T1.item, T2.item
having (1.0*count() / F.numtrans) / I.freq > 1

 * sqlite://
Done.


LHS,RHS,lift
juice,cookies,1.6666666666666665
juice,milk,1.25
milk,cookies,1.25
milk,juice,1.25


### Association rules with two items on the left-hand side

In [33]:
support = .5
print(support)

0.5


#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [34]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
1 rows affected.
Done.


item1,item2,numtrans
juice,milk,3


#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [35]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


LHS1,LHS2,RHS,numtrans,count()
juice,milk,chips,3,1
juice,milk,cookies,3,2
juice,milk,eggs,3,1


#### Add check for lift > 1 to get final result

In [36]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Shop T3, Frequent F, ItemFreq I
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID and T3.item = I.item
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having (1.0*count() / F.numtrans) / I.freq > 1

 * sqlite://
Done.


LHS1,LHS2,RHS,lift
juice,milk,cookies,1.6666666666666665


### <font color = 'green'>**Your Turn - Movies dataset association rules**</font>

#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and confidence thresholds (need not be the same) so the number of association rules is more than 10 but less than 20.


In [43]:
support = 0.03
confidence = 0.6
print(support, confidence)

0.03 0.6


In [44]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, item3, numtrans);
insert into Frequent
select T1.item, T2.item, T3.item, count()
from Movies T1, Movies T2, Movies T3
where T1.TID = T2.TID
and T2.TID = T3.TID
and T1.item < T2.item
and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Movies) > :support;

select count(*) from Frequent;
select * from Frequent;

select F.item1, F.item2, F.item3, T4.item as rhs,
       1.0 * count() / F.numtrans as confidence
from Frequent F, Movies T1, Movies T2, Movies T3, Movies T4
where T1.item = F.item1
and T2.item = F.item2
and T3.item = F.item3
and T1.TID = T2.TID
and T2.TID = T3.TID
and T3.TID = T4.TID
and T4.item not in (F.item1, F.item2, F.item3)
group by F.item1, F.item2, F.item3, T4.item
having confidence > :confidence
order by confidence desc;

 * sqlite://
Done.
Done.
14 rows affected.
Done.
Done.
Done.


item1,item2,item3,rhs,confidence
Boyhood,Fury,Gone Girl,The Imitation Game,0.9069767441860465
Boyhood,Fury,The Imitation Game,Gone Girl,0.9069767441860465
Big Hero 6,Fury,Gone Girl,The Imitation Game,0.8372093023255814
Big Hero 6,Boyhood,Gone Girl,The Imitation Game,0.8214285714285714
Big Hero 6,Fury,The Imitation Game,Gone Girl,0.8181818181818182
Boyhood,Inside Out,The Imitation Game,Gone Girl,0.8076923076923077
Big Hero 6,Boyhood,The Imitation Game,Gone Girl,0.8070175438596491
Boyhood,Gone Girl,Inside Out,The Imitation Game,0.7777777777777778
Big Hero 6,Gone Girl,Inside Out,The Imitation Game,0.6941176470588235
Big Hero 6,Fury,Gone Girl,Inside Out,0.6744186046511628


#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and lift thresholds so the number of association rules is more than 10 but less than 20.


In [61]:
support = 0.03
lift = 32
print(support, lift)

0.03 32


In [63]:
%%sql
drop table if exists ItemFreq;
create table ItemFreq(item, numtrans);
insert into ItemFreq
select item, count(distinct TID)
from Movies
group by item;

select * from ItemFreq;

 * sqlite://
Done.
Done.
123 rows affected.
Done.


item,numtrans
400 Days,3
A Walk in the Woods,1
About Alex,7
Action Jackson,1
American Ultra,18
Annie,6
Anti-Social,1
Appropriate Behaviour,2
Ascension,3
Aziz Ansari: Live at Madison Square Garden,1


In [62]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, item3, numtrans);
insert into Frequent
select T1.item, T2.item, T3.item, count()
from Movies T1, Movies T2, Movies T3
where T1.TID = T2.TID
and T2.TID = T3.TID
and T1.item < T2.item
and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Movies) > :support;


select * from Frequent;

 * sqlite://
Done.
Done.
14 rows affected.
Done.


item1,item2,item3,numtrans
Big Hero 6,Boyhood,Gone Girl,56
Big Hero 6,Boyhood,The Imitation Game,57
Big Hero 6,Fury,Gone Girl,43
Big Hero 6,Fury,The Imitation Game,44
Big Hero 6,Gone Girl,Inside Out,85
Big Hero 6,Gone Girl,The Imitation Game,119
Big Hero 6,Inside Out,The Imitation Game,102
Boyhood,Fury,Gone Girl,43
Boyhood,Fury,The Imitation Game,43
Boyhood,Gone Girl,Inside Out,54


In [64]:
%%sql
select F.item1 || ' | ' || F.item2 || ' | ' || F.item3 || ' -> ' || T4.item as rule,
       (1.0 * count() / F.numtrans) / (I.numtrans * 1.0 / (select count(distinct TID) from Movies)) as lift
from Frequent F, Movies T1, Movies T2, Movies T3, Movies T4, ItemFreq I
where T1.item = F.item1
and T2.item = F.item2
and T3.item = F.item3
and T1.TID = T2.TID
and T2.TID = T3.TID
and T3.TID = T4.TID
and T4.item not in (F.item1, F.item2, F.item3)
and I.item = T4.item
group by F.item1, F.item2, F.item3, T4.item, I.numtrans
having lift > 20
order by lift desc;

 * sqlite://
Done.


rule,lift
Big Hero 6 | Fury | Gone Girl -> Far from Men,32.139534883720934
Big Hero 6 | Fury | Gone Girl -> Flowers in the Attic,32.139534883720934
Big Hero 6 | Fury | Gone Girl -> I'll See You in My Dreams,32.139534883720934
Big Hero 6 | Fury | Gone Girl -> Outcast,32.139534883720934
Big Hero 6 | Fury | Gone Girl -> The Humbling,32.139534883720934
Big Hero 6 | Fury | Gone Girl -> Time Out of Mind,32.139534883720934
Big Hero 6 | Fury | Gone Girl -> Whitey: United States of America v. James J. Bulger,32.139534883720934
Boyhood | Fury | Gone Girl -> Catch Hell,32.139534883720934
Boyhood | Fury | Gone Girl -> Far from Men,32.139534883720934
Boyhood | Fury | Gone Girl -> Flowers in the Attic,32.139534883720934
